# Reddit Climate Change - Modeling & Evaluation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [ ]:
#  Topic detection

# LDA
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
# http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

# BERTopic
# https://maartengr.github.io/BERTopic/index.html


In [2]:
# Preparing environment
%pip install bertopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd

  Using cached bertopic-0.14.1-py2.py3-none-any.whl (120 kB)
  Using cached hdbscan-0.8.29-cp310-cp310-macosx_10_9_universal2.whl
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.2 MB/s eta 0:00:0000:0100:01
  Using cached Cython-0.29.34-py2.py3-none-any.whl (988 kB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached torch-2.0.0-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
  Using cached torchvision-0.15.1-cp310-cp310-macosx_11_0_arm64.whl (1.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.6 MB/s eta 0:00:00
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00a 0:00:01
  Using cached pynndescent-0.5.10-py3-none-any.whl
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
     ━━

/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [5]:
# Load data
clean_posts = pd.read_csv("data/preprocessed_posts.csv")
clean_comments = pd.read_csv("data/preprocessed_comments.csv")

/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_1980/2118955341.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_comments = pd.read_csv("data/preprocessed_comments.csv")


In [7]:
# count NAs in date column
clean_comments["created_date"].isna().sum()

1160540

In [9]:
# Drop NAs, should be done in preprocessing file, delete it here!
clean_comments = clean_comments.dropna(axis=0)

# Modeling Topic Clusters